<a href="https://colab.research.google.com/github/kyrcha/ml-rants/blob/master/xgboost_rbf_bayesian_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing XGBoost and Random Forests with Bayesian Optimiztion

Inspired by [this post](https://www.kdnuggets.com/2019/07/xgboost-random-forest-bayesian-optimisation.html) we will create a full end-to-end pipeline with these three ML algorithms on a regression dataset. More specifically we will use the [Energy efficiency dataset](https://archive.ics.uci.edu/ml/datasets/energy+efficiency) from the UCI repository.

We begin by importing the dataset (after some preprocessing from the original dataset which is in xls format).

In [1]:
import pandas as pd

data = pd.read_csv("ENB2012_data.csv", sep=";")
data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307201,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090196,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


We will split the dataset using a 80-20% split, randomly, keeping 80% for training (with cross validation for tuning hyperparams with Bayesian optimization) and 20% for testing (checking the generalization error).

In [0]:
X = data.drop(labels=['Y1', 'Y2'], axis=1)
y = data['Y2']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

Let's train using CV, a Random Forest regressor with default values.

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

CV_plain_rf_score = abs(cross_val_score(
     RandomForestRegressor(random_state=42),  
     X=X_train, 
     y=y_train, 
     cv=10,
     scoring="neg_mean_squared_error",
     n_jobs=-1).mean())

model_plain_rf = RandomForestRegressor(n_jobs=-1, random_state=42)
model_plain_rf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [5]:
from sklearn.metrics import mean_squared_error

pred_train = model_plain_rf.predict(X_train)
pred_test = model_plain_rf.predict(X_test)

print("Training MSE: {0:.2f}".format(mean_squared_error(y_train, pred_train)))
print("CV MSE: {0:.2f}".format(abs(CV_plain_rf_score)))
print("Testing MSE: {0:.2f}".format(mean_squared_error(y_test, pred_test)))

Training MSE: 0.48
CV MSE: 3.14
Testing MSE: 3.65


Let's also check the XGBoost model with default parameters.

In [6]:
import xgboost as xgb

CV_xgb_plain_score_ = abs(cross_val_score(
     xgb.XGBRegressor(random_state=42),  
     X=X_train, 
     y=y_train, 
     cv=10,
     scoring="neg_mean_squared_error",
     n_jobs=-1).mean())


model_plain_xgb = xgb.XGBRegressor(random_state=42)
model_plain_xgb.fit(X_train, y_train)

[14:07:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [7]:
pred_train = model_plain_xgb.predict(X_train)
pred_test = model_plain_xgb.predict(X_test)

print("Training MSE: {0:.2f}".format(mean_squared_error(y_train, pred_train)))
print("CV MSE: {0:.2f}".format(abs(CV_xgb_plain_score_)))
print("Testing MSE: {0:.2f}".format(mean_squared_error(y_test, pred_test)))

Training MSE: 1.82
CV MSE: 2.36
Testing MSE: 2.50


## Bayesian Optimization

We will begin by installing the [Bayesian optimization package](https://github.com/fmfn/BayesianOptimization) and then creating functions to optimize the hyperparameters of the two algorithms.

In [8]:
!pip install bayesian-optimization

  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [0]:
from bayes_opt import BayesianOptimization

def bayesian_optimization(dataset, function, parameters):
   X_train, y_train = dataset
   n_iterations = 10
   gp_params = {"alpha": 1e-4}

   BO = BayesianOptimization(function, parameters)
   BO.maximize(n_iter=n_iterations, **gp_params)

   return BO.max

In [10]:
def rfc_optimization(cv_splits):
    def function(n_estimators, max_depth, min_samples_split):
        return cross_val_score(
               RandomForestRegressor(
                   n_estimators=int(max(n_estimators,0)),                                                               
                   max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)), 
                   n_jobs=-1, 
                   random_state=42),  
               X=X_train, 
               y=y_train, 
               cv=cv_splits,
               scoring="neg_mean_squared_error",
               n_jobs=-1).mean()

    parameters = {"n_estimators": (10, 1000),
                  "max_depth": (1, 150),
                  "min_samples_split": (2, 10)}
    
    return function, parameters
  
f, p = rfc_optimization(10)
dataset = (X_train, y_train)
best_solution = bayesian_optimization(dataset, f, p) 

|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        | -2.931    |  31.4     |  3.828    |  216.6    |
|  2        | -3.131    |  125.3    |  5.407    |  108.7    |
|  3        | -3.116    |  80.17    |  5.37     |  102.3    |
|  4        | -2.881    |  147.5    |  2.498    |  762.5    |
|  5        | -3.199    |  108.9    |  9.787    |  47.21    |
|  6        | -17.93    |  1.0      |  10.0     |  1e+03    |
|  7        | -3.209    |  94.54    |  10.0     |  196.3    |
|  8        | -2.888    |  150.0    |  2.0      |  693.6    |
|  9        | -3.187    |  80.65    |  10.0     |  710.2    |
|  10       | -2.854    |  40.78    |  2.0      |  316.0    |
|  11       | -2.865    |  144.9    |  2.0      |  349.5    |
|  12       | -2.953    |  82.02    |  3.694    |  454.2    |
|  13       | -3.187    |  150.0    |  10.0     |  545.3    |
|  14       | -17.93    |  1.0      |  2.0      |  561.1    |
|  15   

Based on this optimization procedure we have:

*  n_estimators = 316
*  min_samples_split = 2
*  max_depth = 40



In [12]:
params = best_solution["params"]

model = RandomForestRegressor(
             n_estimators=int(max(params["n_estimators"], 0)),
             max_depth=int(max(params["max_depth"], 1)),
             min_samples_split=int(max(params["min_samples_split"], 2)), 
             n_jobs=-1, 
             random_state=42)

model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=316, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [14]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

print("Training MSE: {0:.2f}".format(mean_squared_error(y_train, pred_train)))
print("CV MSE: {0:.2f}".format(2.854))
print("Testing MSE: {0:.2f}".format(mean_squared_error(y_test, pred_test)))

Training MSE: 0.37
CV MSE: 2.85
Testing MSE: 3.05


In [16]:
def xgb_optimization(cv_splits):
    def function(eta, gamma, max_depth):
            return cross_val_score(
                   xgb.XGBRegressor(
                       objective="reg:squarederror",
                       learning_rate=max(eta, 0),
                       gamma=max(gamma, 0),
                       max_depth=int(max_depth),                                               
                       seed=42,
                       nthread=-1),  
                   X=X_train, 
                   y=y_train, 
                   cv=cv_splits,
                   scoring="neg_mean_squared_error",
                   n_jobs=-1).mean()

    parameters = {"eta": (0.001, 0.4),
                  "gamma": (0, 20),
                  "max_depth": (1, 2000)}
    
    return function, parameters
  

f, p = xgb_optimization(10)
dataset = (X_train, y_train)
best_solution = bayesian_optimization(dataset, f, p) 


|   iter    |  target   |    eta    |   gamma   | max_depth |
-------------------------------------------------------------
|  1        | -1.467    |  0.2486   |  3.14     |  1.485e+0 |
|  2        | -1.499    |  0.2835   |  8.228    |  471.5    |
|  3        | -1.895    |  0.3508   |  11.09    |  5.861    |
|  4        | -1.356    |  0.1632   |  1.355    |  91.55    |
|  5        | -2.137    |  0.06769  |  16.84    |  1.966e+0 |
|  6        | -544.1    |  0.001    |  0.0      |  1.041e+0 |
|  7        | -1.651    |  0.2585   |  6.305    |  49.36    |
|  8        | -1.319    |  0.1819   |  2.295    |  78.51    |
|  9        | -1.646    |  0.3446   |  10.87    |  23.62    |
|  10       | -544.1    |  0.001    |  5.579    |  84.03    |
|  11       | -2.03     |  0.1575   |  17.66    |  357.7    |
|  12       | -1.908    |  0.1076   |  12.11    |  725.3    |
|  13       | -2.034    |  0.1213   |  16.94    |  558.9    |
|  14       | -1.571    |  0.1449   |  6.503    |  1.634e+0 |
|  15   

For XGBoost we have the optimal parameters found:

*  max_depth = 78
*  gamma = 2.295
*  eta = 0.1819

In [17]:
params = best_solution["params"]

model = xgb.XGBRegressor(objective="reg:squarederror",
                 learning_rate=max(params["eta"], 0),
                 gamma=max(params["gamma"], 1),
                 max_depth=int(max(params["max_depth"], 2)),                                               
                 seed=42,
                 nthread=-1)

model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=2.295287077901593,
             importance_type='gain', learning_rate=0.18193061733363647,
             max_delta_step=0, max_depth=78, min_child_weight=1, missing=None,
             n_estimators=100, n_jobs=1, nthread=-1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=42, silent=None,
             subsample=1, verbosity=1)

In [18]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

print("Training MSE: {0:.2f}".format(mean_squared_error(y_train, pred_train)))
print("CV MSE: {0:.2f}".format(1.319))
print("Testing MSE: {0:.2f}".format(mean_squared_error(y_test, pred_test)))

Training MSE: 0.25
CV MSE: 1.32
Testing MSE: 1.34


## Final Results

To conclude we constructed the following matrix for the CV and test errors:

|  Model | CV MSE | Test MSE  | 
|---|---|---|
| RF default params |   3.14 | 3.65  | 
| XGBoost default params  |  2.36 | 2.50  | 
| RF optimized params  | 2.85  | 3.05  | 
| XGBoost optimized params  | 1.32  | 1.34  | 

Bayesian optimization indeed provided boost in performance in a more "clever" manner than grid or random search.
